In [10]:
import os
import numpy as np
import pydicom
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from pydicom.pixels import iter_pixels
import cv2

# VGG16 Modeli
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(240, 240, 3))
vgg16.trainable = False

# Üstüne yeni katmanlar ekleyelim
model = models.Sequential()
model.add(vgg16)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Klasörler
folders = ["Tcga_lgg_olio_filtered", "Tcga_lgg_astro_filtered", "Tcga_gbm_filtered"]
labels = {"Tcga_lgg_olio_filtered": 0, "Tcga_lgg_astro_filtered": 1, "Tcga_gbm_filtered": 2}

# Eğitim Parametreleri
batch_size = 200
batch_data = []
batch_labels = []

# Eğitim Başlıyor
for folder in folders:
   print(f"Processing folder: {folder}")
   label = labels[folder]

   for file in os.listdir(folder):
        
      dcm_path = os.path.join(folder, file)
           
      dcm = pydicom.dcmread(dcm_path)
            
      for arr in iter_pixels(dcm_path):
               
            
                
               if arr.shape[0] == 256:
                  start_x = (256 - 240) // 2
                  start_y = (256 - 240) // 2


                  arr = arr[start_y:start_y+240, start_x:start_x+240]
               
               # Normalize et ve batch'e ekle
               arr = arr / 255.0
               batch_data.append(arr)
               batch_labels.append(label)

               # Batch dolunca eğit
               if len(batch_data) == batch_size:
                  print(f"Training on batch of size {len(batch_data)}")
                  nmpy_batch_data = np.array(batch_data)
                  nmpy_batch_labels = to_categorical(batch_labels, num_classes=3)
                  model.fit(nmpy_batch_data, nmpy_batch_labels, epochs=1, verbose=1)
                  batch_data.clear()
                  batch_labels.clear()
                  

      if batch_data:
                  
         print(f"Training on final batch of size {len(batch_data)}")
         nmpy_batch_data = np.array(batch_data)
         nmpy_batch_labels = to_categorical(batch_labels, num_classes=3)
         model.fit(nmpy_batch_data, nmpy_batch_labels, epochs=1, verbose=1)     
                  

# Son kalan veriyi eğit


print("Training completed!")

Processing folder: Tcga_lgg_olio_filtered
Training on batch of size 200


2025-03-26 13:03:18.226196: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 471859200 exceeds 10% of free system memory.
2025-03-26 13:03:18.477105: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 471859200 exceeds 10% of free system memory.
2025-03-26 13:03:18.988125: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 235929600 exceeds 10% of free system memory.
2025-03-26 13:03:19.180050: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 235929600 exceeds 10% of free system memory.


1/7 ━━━━━━━━━━━━━━━━━━━━ 24s 4s/step - accuracy: 0.2188 - loss: 1.4755

2025-03-26 13:03:21.283386: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 471859200 exceeds 10% of free system memory.


7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.7295 - loss: 0.5109
Training on batch of size 200
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00
Training on batch of size 200
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00
Training on batch of size 200
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00
Training on batch of size 200
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00
Training on batch of size 200
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00
Training on batch of size 200
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00
Training on batch of size 200
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00
Training on batch of size 200
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00
Training on batch of size 200
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 1.0000 - l

In [12]:
import os
import numpy as np
import pydicom
from tensorflow.keras.utils import Sequence, to_categorical
import cv2

class DicomDataGenerator(Sequence):
    def __init__(self, folders, labels, batch_size=32, input_size=(240, 240), num_classes=3, shuffle=True):
        self.folders = folders
        self.labels = labels
        self.batch_size = batch_size
        self.input_size = input_size
        self.num_classes = num_classes
        self.shuffle = shuffle

        # Tüm dosya yollarını al
        self.file_paths = []
        for folder in folders:
            folder_path = os.path.join(folder)
            files = os.listdir(folder_path)
            self.file_paths.extend([(os.path.join(folder_path, f), labels[folder]) for f in files])
        
        if self.shuffle:
            np.random.shuffle(self.file_paths)
    
    def __len__(self):
        # Kaç adımda bir epoch tamamlanacak
        return int(np.ceil(len(self.file_paths) / self.batch_size))
    
    def __getitem__(self, index):
        # Batch için dosya yollarını seç
        batch_paths = self.file_paths[index * self.batch_size: (index + 1) * self.batch_size]
        
        batch_data = []
        batch_labels = []

        for path, label in batch_paths:
            # DICOM dosyasını yükle
            dcm = pydicom.dcmread(path)
            arr = dcm.pixel_array

            # Görüntü boyutunu ayarla
            arr = cv2.resize(arr, self.input_size)
            
            # Normalize et ve 3 kanallı hale getir (VGG16 için)
            arr = arr.astype(np.float32) / np.max(arr)
            arr = np.stack([arr] * 3, axis=-1)

            batch_data.append(arr)
            batch_labels.append(label)
        
        # Numpy array'e çevir
        batch_data = np.array(batch_data)
        batch_labels = to_categorical(batch_labels, num_classes=self.num_classes)

        return batch_data, batch_labels
    
    def on_epoch_end(self):
        # Her epoch sonunda veriyi karıştır
        if self.shuffle:
            np.random.shuffle(self.file_paths)


In [14]:
import os
import numpy as np
import pydicom
from tensorflow.keras.utils import Sequence, to_categorical
import cv2

class DicomDataGenerator(Sequence):
    def __init__(self, folders, labels, batch_size=32, input_size=(240, 240), num_classes=3, shuffle=True):
        self.folders = folders
        self.labels = labels
        self.batch_size = batch_size
        self.input_size = input_size
        self.num_classes = num_classes
        self.shuffle = shuffle

        # Tüm dosya yollarını al
        self.file_paths = []
        for folder in folders:
            folder_path = os.path.join(folder)
            files = os.listdir(folder_path)
            self.file_paths.extend([(os.path.join(folder_path, f), labels[folder]) for f in files])
        
        if self.shuffle:
            np.random.shuffle(self.file_paths)
    
    def __len__(self):
        # Kaç adımda bir epoch tamamlanacak (Toplam slice sayısına bağlı)
        total_slices = sum(pydicom.dcmread(path).pixel_array.shape[0] for path, _ in self.file_paths)
        return int(np.ceil(total_slices / self.batch_size))
    
    def __getitem__(self, index):
        batch_data = []
        batch_labels = []

        slice_count = 0
        for path, label in self.file_paths:
            # DICOM dosyasını yükle
            dcm = pydicom.dcmread(path)
            slices = dcm.pixel_array  # Shape: (slice, 256, 256, 3)
            
            for i in range(slices.shape[0]):
                arr = slices[i]

                # Görüntü boyutunu ayarla
                arr = cv2.resize(arr, self.input_size)

                # Normalize et ve doğru şekle getir
                arr = arr.astype(np.float32) / np.max(arr)

                # Eğer zaten 3 kanallıysa, direkt ekle; değilse çoğalt
                if arr.shape[-1] != 3:
                    arr = np.stack([arr] * 3, axis=-1)

                batch_data.append(arr)
                batch_labels.append(label)
                slice_count += 1

                # Batch dolduğunda çık
                if slice_count >= self.batch_size:
                    break

            # Batch dolduysa dışarı çık
            if slice_count >= self.batch_size:
                break

        # Numpy array'e çevir
        batch_data = np.array(batch_data)
        batch_labels = to_categorical(batch_labels, num_classes=self.num_classes)

        return batch_data, batch_labels
    
    def on_epoch_end(self):
        # Her epoch sonunda veriyi karıştır
        if self.shuffle:
            np.random.shuffle(self.file_paths)


In [ ]:
# Klasörler ve etiketler
folders = ["Tcga_lgg_olio_filtered", "Tcga_lgg_astro_filtered", "Tcga_gbm_filtered"]
labels = {"Tcga_lgg_olio_filtered": 0, "Tcga_lgg_astro_filtered": 1, "Tcga_gbm_filtered": 2}



from tensorflow.keras.optimizers import Adam

# Learning rate'i baştan düşük ayarla
optimizer = Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Data Generator oluştur
train_generator = DicomDataGenerator(folders, labels, batch_size=32)

# Modeli eğit
model.fit(train_generator, epochs=10, verbose=1)

print("Eğitim tamamlandı!")


Epoch 1/10
952/952 ━━━━━━━━━━━━━━━━━━━━ 3061s 3s/step - accuracy: 0.8318 - loss: 11.1750
Epoch 2/10
952/952 ━━━━━━━━━━━━━━━━━━━━ 3253s 3s/step - accuracy: 0.8194 - loss: 2.2326
Epoch 3/10
291/952 ━━━━━━━━━━━━━━━━━━━━ 41:44 4s/step - accuracy: 1.0000 - loss: 0.0592